In [12]:
import pandas as pd
from time import time
import os, sys
from datetime import datetime as dt

In [13]:
sys.path.append('/Users/pratyush/Desktop/Assignments/dsf/project/venv/lib/python3.7/site-packages')

In [195]:
from pytrends.request import TrendReq

In [300]:
def getRawTrends(entityList):
    pt = TrendReq(hl='en-US', tz=360, timeout=(100,250), retries=2, backoff_factor=1.0)
    pt.build_payload(
        entityList,
        timeframe='all'
        #cat=34
    )
    return pt.interest_over_time()

def makeDataframe(entityList):
    start = time()
    curr = getRawTrends(entityList)
    timeframe = list(curr.index)
    timeList = timeframe
    moviesList = []
    trendList = []
    raw = {'movie': timeframe}
    for i in entityList:
        raw[i] = curr[i]
    df = pd.DataFrame(raw)
    df_grouped = df.groupby(df['movie'].map(lambda x: x.year)).sum()
    end = time()
    print(end - start)
    return df_grouped.T

def getTrends(entityList):
    df = makeDataframe(entityList)
    return df

In [301]:
df = getTrends(['Joker', 'The Dark Knight', 'Die Hard', 'Superman', 'Avengers: Endgame'])

0.834204912185669


In [302]:
df

movie,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Joker,27,31,28,35,78,61,54,55,76,72,73,100,116,92,86,232
The Dark Knight,0,0,0,16,85,18,12,36,96,30,23,20,15,12,12,12
Die Hard,12,12,13,41,14,12,12,13,14,34,14,13,14,13,14,11
Superman,93,113,195,100,86,80,87,99,93,134,128,156,255,98,82,64
Avengers: Endgame,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,146
